In [21]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import time
from prettytable import PrettyTable
from sklearn.ensemble import RandomForestClassifier

# Implementace DecisionTree a RandomForest

In [22]:
class DecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        self.n_classes = len(set(y))
        self.n_features = X.shape[1]
        self.tree = self._grow_tree(X, y)

    def predict(self, X):
        return np.array([self._predict(inputs) for inputs in X])

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        if n_samples <= 1 or depth >= self.max_depth or len(set(y)) == 1:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)

        best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def _best_criteria(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)
            for threshold in thresholds:
                gain = self._information_gain(y, X_column, threshold)
                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = threshold
        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        parent_entropy = self._entropy(y)
        left_idxs, right_idxs = self._split(X_column, split_thresh)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r
        ig = parent_entropy - child_entropy
        return ig

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        if len(y) == 0:
            return 0
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log2(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def _predict(self, inputs):
        node = self.tree
        while node.left or node.right:
            node = node.left if inputs[node.feature] <= node.threshold else node.right
        return node.value

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class RandomForest:
    def __init__(self, n_trees=100, max_depth=None, n_features=None, random_state=None):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.n_features = n_features
        self.random_state = random_state
        self.trees = []
        if random_state:
            np.random.seed(random_state)

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(max_depth=self.max_depth)
            X_sample, y_sample = self._bootstrap_sample(X, y)
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [self._most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

    def _bootstrap_sample(self, X, y):
        if X.shape[0] == 0 or len(y) == 0:
            raise ValueError("Bootstrap sample failed due to empty dataset.")
        n_samples = X.shape[0]
        idxs = np.random.choice(n_samples, n_samples, replace=True)
        return X[idxs], y[idxs]

    def _most_common_label(self, y):
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

# Načtení základního datasetu pro testování implementace

In [23]:
titanic = pd.read_csv('titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing
- jednoduchý preprocessing v rámci datasetu `titanic.csv`, aby byl vhodný pro klasifikační testování.


In [24]:
titanic = titanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'])

In [25]:
titanic.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

- doplnění chybějících hodnot v příslušných sloupcích a následný encoding s pomocí **LabelEncoder**
- vytvoření target proměnné `Survived` a vytvoření `train_test_split`

In [26]:
titanic.fillna({'Age': titanic['Age'].mean()}, inplace=True)
titanic.fillna({'Embarked':titanic['Embarked'].mode()[0]}, inplace=True)

label_encoder = LabelEncoder()
titanic['Sex'] = label_encoder.fit_transform(titanic['Sex'])
titanic['Embarked'] = label_encoder.fit_transform(titanic['Embarked'])


X_titanic = titanic.drop(columns=['Survived']).values
y_titanic = titanic['Survived'].values

X_train_titanic, X_test_titanic, y_train_titanic, y_test_titanic = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42)

# Testování vlastní implementace RandomForest algoritmu

In [27]:
start_fit = time.time()
rf_titanic = RandomForest(n_trees=100, max_depth=10, random_state=42)
rf_titanic.fit(X_train_titanic, y_train_titanic)
end_fit = time.time()

start_predict = time.time()
y_pred_titanic = rf_titanic.predict(X_test_titanic)
end_predict = time.time()

accuracy_titanic = accuracy_score(y_test_titanic, y_pred_titanic)
precision_titanic = precision_score(y_test_titanic, y_pred_titanic, average='binary')
recall_titanic = recall_score(y_test_titanic, y_pred_titanic, average='binary')
f1_titanic = f1_score(y_test_titanic, y_pred_titanic, average='binary')

fit_time = end_fit - start_fit
predict_time = end_predict - start_predict
total_time = fit_time + predict_time

table_metrics = PrettyTable()
table_metrics.field_names = ["Metric", "Value"]
table_metrics.add_row(["Accuracy", accuracy_titanic])
table_metrics.add_row(["Precision", precision_titanic])
table_metrics.add_row(["Recall", recall_titanic])
table_metrics.add_row(["F1 Score", f1_titanic])

table_time = PrettyTable()
table_time.field_names = ["Stage", "Time (seconds)"]
table_time.add_row(["Fit Time", fit_time])
table_time.add_row(["Predict Time", predict_time])
table_time.add_row(["Total Time", total_time])

print("Titanic Dataset Performance Metrics:")
print(table_metrics)
print("\nTime Analysis:")
print(table_time)

print("\nClassification Report:")
print(classification_report(y_test_titanic, y_pred_titanic))

Titanic Dataset Performance Metrics:
+-----------+--------------------+
|   Metric  |       Value        |
+-----------+--------------------+
|  Accuracy | 0.8156424581005587 |
| Precision | 0.847457627118644  |
|   Recall  | 0.6756756756756757 |
|  F1 Score | 0.7518796992481203 |
+-----------+--------------------+

Time Analysis:
+--------------+----------------------+
|    Stage     |    Time (seconds)    |
+--------------+----------------------+
|   Fit Time   |  17.093661308288574  |
| Predict Time | 0.031196117401123047 |
|  Total Time  |  17.124857425689697  |
+--------------+----------------------+

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.91      0.85       105
           1       0.85      0.68      0.75        74

    accuracy                           0.82       179
   macro avg       0.82      0.79      0.80       179
weighted avg       0.82      0.82      0.81       179



# Testování implementace RandomForest algoritmu z knihovny sklearn

In [28]:
start_fit = time.time()
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_clf.fit(X_train_titanic, y_train_titanic)
end_fit = time.time()

start_predict = time.time()
y_pred_titanic_clf = rf_clf.predict(X_test_titanic)
end_predict = time.time()

accuracy_titanic_clf = accuracy_score(y_test_titanic, y_pred_titanic_clf)
precision_titanic_clf = precision_score(y_test_titanic, y_pred_titanic_clf, average='binary')
recall_titanic_clf = recall_score(y_test_titanic, y_pred_titanic_clf, average='binary')
f1_titanic_clf = f1_score(y_test_titanic, y_pred_titanic_clf, average='binary')

fit_time = end_fit - start_fit
predict_time = end_predict - start_predict
total_time = fit_time + predict_time

table_metrics = PrettyTable()
table_metrics.field_names = ["Metric", "Value"]
table_metrics.add_row(["Accuracy", accuracy_titanic_clf])
table_metrics.add_row(["Precision", precision_titanic_clf])
table_metrics.add_row(["Recall", recall_titanic_clf])
table_metrics.add_row(["F1 Score", f1_titanic_clf])

table_time = PrettyTable()
table_time.field_names = ["Stage", "Time (seconds)"]
table_time.add_row(["Fit Time", fit_time])
table_time.add_row(["Predict Time", predict_time])
table_time.add_row(["Total Time", total_time])

print("Titanic Dataset Performance Metrics:")
print(table_metrics)
print("\nTime Analysis:")
print(table_time)

print("\nClassification Report:")
print(classification_report(y_test_titanic, y_pred_titanic_clf))

Titanic Dataset Performance Metrics:
+-----------+--------------------+
|   Metric  |       Value        |
+-----------+--------------------+
|  Accuracy | 0.8268156424581006 |
| Precision | 0.8412698412698413 |
|   Recall  | 0.7162162162162162 |
|  F1 Score | 0.7737226277372263 |
+-----------+--------------------+

Time Analysis:
+--------------+----------------------+
|    Stage     |    Time (seconds)    |
+--------------+----------------------+
|   Fit Time   |  0.1494295597076416  |
| Predict Time | 0.006574153900146484 |
|  Total Time  | 0.15600371360778809  |
+--------------+----------------------+

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.90      0.86       105
           1       0.84      0.72      0.77        74

    accuracy                           0.83       179
   macro avg       0.83      0.81      0.82       179
weighted avg       0.83      0.83      0.82       179



# Závěr

## Srovnání výkonu vlastní implementace Random Forest a RandomForestClassifier

### Metriky měření přesnosti v %

| Metrika        | Vlastní implementace | Scikit-learn implementace |
|----------------|----------------------|---------------------------|
| **Accuracy**   | 81.56 %             | 82.68 %                  |
| **Precision** | 84.75 %             | 84.13 %                  |
| **Recall**   | 67.57 %             | 71.62 %                  |
| **F1 Score**   | 75.19 %             | 77.37 %                  |

**Závěr:** 
- Scikit-learn implementace dosahuje o něco lepších výsledků ve všech metrikách, zejména v Recall a F1 score.

---

### Časová náročnost pro počet stromů 100 a hloubka stromů 10

| Fáze           | Vlastní implementace (s) | Scikit-learn implementace (s) |
|----------------|--------------------------|-------------------------------|
| **Trénování**  | 17.06                   | 0.14                          |
| **Predikce**   | 0.03                    | 0.006                         |
| **Celkem**     | 17.09                   | 0.15                          |

**Závěr:** 
- Scikit-learn implementace je výrazně rychlejší při trénování (více než 100x rychlejší) i při predikci.

---

### Shrnutí

1. **Výkon metrik:** Scikit-learn implementace poskytuje lepší přesnost, recall a F1 skóre, avšak vhledem k rozdílu v komplexnosti implementace a možnosti nastavení volitelných parametrů u verze z knihovny, lze konstatovat, že přesnost vlastní implementace je do jisté míry uspokojivá.  
2. **Rychlost:** Scikit-learn implementace je výrazně rychlejší díky optimalizacím knihovny a paralelnímu zpracování.  
3. **Možnosti vylepšení vlastní implementace:**  
   - **Optimalizace výběru vlastností a prahových hodnot:**  
     Místo výpočtu všech prahů pomocí `np.unique` lze zkusit vzorkování prahů nebo zmenšit počet zkoumaných prahů pomocí binování dat.
   - **Vektorové výpočty:**  
     Přepsání výpočtů entropie, informačního zisku a rozdělení na vektorovou podobu s využitím NumPy.
   - **Parallelizace:**  
     Využití paralelního zpracování stromů pomocí knihovny `joblib` nebo `concurrent.futures` k urychlení trénování Random Forest.
   - **Efektivní bootstrap sampling:**  
     Nahrazení `np.random.choice` efektivnějším `np.random.randint`, aby se minimalizovalo zpracování velkých datasetů.
   - **Optimalizace stromové struktury:**  
     Ukládání dat stromu jako NumPy pole místo rekurzivní struktury uzlů ke zlepšení přístupové rychlosti.
   - **Použití nízkoúrovňové optimalizace:**  
     Přepsání kritických částí pomocí Cython nebo Numba pro rychlejší výpočty.

